In [1]:
from IPython.display import HTML
HTML(open("../style.css", "r").read())

If `nb-mypy` isn't installed, run:
```
!pip install nb-mypy 
```

In [2]:
%load_ext nb_mypy
%nb_mypy On

Version 1.0.6


# Spam Detection  Using the Naive Bayes Algorithm

The process of creating a spam detector using the naive Bayes algorithm is split up into four steps.

  - Create a set of the most common words occurring in spam and ham (i.e. non-spam) emails.
  - For every word occurring in this set, compute the 
  probability that this word occurs in a spam or ham email.
  - Create a function that takes an email and the conditional probabilities computed before and that then computes the probability
    that the given email is spam.
  - Evaluate the <em style='color:blue;'>precision</em> and the <em style='color:blue;'>recall</em> of the spam classifier.

## Step 1: Create Word Dictionary

We need the module `os` for reading directories and the module `re` for 
<em style='color:blue;'>regular expressions</em>.

In [3]:
import os
import re
import math

An object of class <a href='https://docs.python.org/2/library/collections.html#counter-objects'>`Counter`</a> is a special form of a `dictionary` that is used for counting.  We need a counter to figure out what the most common words are.

In [4]:
from collections import Counter

The constructor `Counter` can be called with an iterable (e.g. a list, a set, or a string) as its argument.
It returns a dictionary where the values are the number of occurrences.  The example below counts characters in a string.

In [5]:
Cntr = Counter("abracadabra")
Cntr

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

An important method of the class `Counter` is `update`.  If `Ctr` is a counter and `S` is an iterable, then the elements of `S` are added to `Ctr`. If an element `e` occurs `k` times in `S`, then 
the count of `e` in `Ctr` is incremented by `k`.

In [38]:
Cntr.update("abba")
Cntr

Counter({'a': 9, 'b': 6, 'r': 2, 'c': 1, 'd': 1})

### The Email Data

The directory 
https://github.com/karlstroetmann/Artificial-Intelligence/tree/master/Python/6%20Classification/EmailData
contains 960 emails that are divided into four subdirectories:

  - `spam-train` contains 350 spam emails for training,
  - `ham-train`  contains 350 ham (i.e. non-spam) emails for training,
  - `spam-test`  contains 130 spam emails for testing,
  - `ham-test`   contains 130 ham  emails for testing.

Originally, this data has been collected by **Ion Androutsopoulos**.  I have found this data on a now defunct 
*open classroom* page on https://online.stanford.edu/free-courses provided by Andrew Ng.

We declare some variables so this notebook can be adapted to other data sets.

In [8]:
spam_dir_train: str = 'EmailData/spam-train/'
ham__dir_train: str = 'EmailData/ham-train/'
spam_dir_test:  str = 'EmailData/spam-test/'
ham__dir_test:  str = 'EmailData/ham-test/'
Directories: list[str] = [spam_dir_train, ham__dir_train, spam_dir_test, ham__dir_test]

In order to compute the <em style='color:blue;'>prior probability</em> that an email is ham or spam we need to count the number of spam and ham emails.

In [9]:
no_spam:    int   = len(os.listdir(spam_dir_train)) # number of spam mails
no_ham:     int   = len(os.listdir(ham__dir_train)) # number of ham  mails
spam_prior: float = no_spam / (no_spam + no_ham)   # probability of a spam mail
ham__prior: float = no_ham  / (no_spam + no_ham)   # probability of a ham mail
spam_prior, ham__prior

(0.5, 0.5)

I have checked that the proportion of spam and ham emails in the test directory is also $1:1$.  If the proportion of spam and ham emails in real life is different from $1:1$, then we would have to use this proportion in the spam filter to be developed.

The function $\texttt{get_words}(\texttt{fn})$ takes a filename $\texttt{fn}$ as its argument.  It reads the file and returns a set of all words that are found in this file.  
The words are then transformed to lower case. Since we use a set, words occurring multiple times are only counted once.

In [10]:
def get_words(fn: str) -> set[str]:
    with open(fn, 'r') as file:
        text: str = file.read().lower()
        return set(re.findall(r"[\w']+", text))

Let us test this function with a small example mail.

In [11]:
cat EmailData/ham-train/3-380msg4.txt || or EmailData/ham-train/3-380msg4.txt

posting hi m work phonetics project modern irish m hard source anyone recommend book article english specifically interest palatal slender consonant work helpful too thank laurel sutton sutton garnet berkeley edu 

In [12]:
get_words('EmailData/ham-train/3-380msg4.txt')

{'anyone',
 'article',
 'berkeley',
 'book',
 'consonant',
 'edu',
 'english',
 'garnet',
 'hard',
 'helpful',
 'hi',
 'interest',
 'irish',
 'laurel',
 'm',
 'modern',
 'palatal',
 'phonetics',
 'posting',
 'project',
 'recommend',
 'slender',
 'source',
 'specifically',
 'sutton',
 'thank',
 'too',
 'work'}

The function `read_all_files` reads all files contained in those directories that are stored in the list `Directories`. 
It returns a `Counter`.  For every word $w$ this counter contains the number of files that contain the word $w$. 

In [13]:
def read_all_files(Directories: list[str]) -> Counter:
    Words: Counter = Counter()
    for directory in Directories:
        for file_name in os.listdir(directory):
            Words.update(get_words(directory + file_name))
    return Words

`Word_Counter` is a dictionary containing all words together with their counts.

In [14]:
Word_Counter: Counter = read_all_files(Directories)
Word_Counter

Counter({'s': 560,
         'information': 448,
         'please': 445,
         'email': 429,
         'one': 404,
         'http': 399,
         'address': 379,
         'our': 365,
         'send': 360,
         'include': 354,
         'mail': 350,
         'call': 347,
         'language': 335,
         'list': 329,
         'follow': 320,
         'us': 308,
         'university': 307,
         'work': 304,
         'free': 302,
         'www': 296,
         'e': 294,
         'name': 289,
         'interest': 283,
         'receive': 283,
         'first': 274,
         'order': 271,
         'here': 259,
         'com': 257,
         'available': 254,
         'over': 250,
         'need': 250,
         'number': 248,
         'fax': 247,
         'day': 244,
         'many': 244,
         'linguistic': 243,
         'most': 232,
         'want': 231,
         'offer': 229,
         'program': 226,
         'nt': 222,
         'question': 221,
         'state': 220,
         's

The email contains 22770 different words. 

In [15]:
len(Word_Counter)

22770

We investigate how many words only occur in a single email.

It wouldn't make sense to use a word as a feature if it only occurs in a single mail.
Let's check how many such word exist.

In [16]:
SingleWords = { w for w in Word_Counter if Word_Counter[w] <= 1 }
SingleWords

{'cuban',
 'tyranny',
 'kachru',
 'btinternet',
 'papersstyle',
 'cybersearch',
 'ryoichus',
 'reflexo',
 'standardizer',
 'whore',
 'visibility',
 'picker',
 'centrum',
 'rubino',
 'lest',
 'probe',
 'distinto',
 'nest',
 'vario',
 'yuganov',
 'jcdcc',
 'newborn',
 'zarautz',
 'kasevich',
 'oksalum',
 'mbuggy',
 'vertically',
 'funicular',
 'heal',
 'nat',
 'verdus',
 'polguere',
 'antics',
 'formelle',
 'daidb',
 'neijt',
 'bilkent',
 'noice',
 'lorong',
 'samba',
 'enrique',
 'feride',
 'mohame',
 'as',
 'parseval',
 'midst',
 'subtlety',
 'resurgence',
 'contingent',
 'mcnally',
 'cyberdrive',
 'blocker',
 'eskerrik',
 'beom',
 'prevalent',
 'beguinage',
 'nominalkomposita',
 'navajo',
 'kln',
 'agustus',
 'budwig',
 'buyout',
 'ehess',
 'sdnt',
 'uwo',
 'marrit',
 'receipient',
 'displace',
 'theorize',
 'farka',
 'outfit',
 'pulling',
 'climate',
 'lad',
 'majordomo',
 'dernier',
 'eire',
 'fade',
 'transferability',
 'mfuxphc',
 'semilattice',
 'ccccc',
 'jollum',
 'hrun',
 'mis

In [17]:
len(SingleWords)

13028

`Common_Words` is a list of those words that occur in at least 10 of our emails.
The number `min_occurrences` is a *hyper parameter* that should be validated via a validation set. 

In [18]:
min_occurrences = 10
Common_Words: set[str] = { w for w in Word_Counter if Word_Counter[w] >= min_occurrences }

There a 2653 words that occur at least `min_occurrences` times.

In [19]:
len(Common_Words)

2653

## Computing the Conditional Probabilities

Having computed the most common words, we are now ready to compute the conditional probability that a given word occurs in a spam email.

The function $\texttt{get_common_words}(\texttt{fn})$ takes a filename $\texttt{fn}$ 
as its argument.  It reads the file and returns the set of all words in `Common_Words` that are found in the given file.  

In [20]:
def get_common_words(fn: str) -> set[str]:
    return get_words(fn) & Common_Words

We test this function for a small email.

In [21]:
get_common_words('EmailData/ham-train/3-380msg4.txt')

{'anyone',
 'article',
 'berkeley',
 'book',
 'consonant',
 'edu',
 'english',
 'hard',
 'helpful',
 'hi',
 'interest',
 'm',
 'modern',
 'phonetics',
 'project',
 'recommend',
 'source',
 'specifically',
 'thank',
 'too',
 'work'}

The function `count_common_words` takes a string specifying a `directory`.  It returns a 
`Counter` that counts how often the words in `Common_Words` occur in any of the files in `directory`.

In [22]:
def count_commmon_words(directory: str) -> Counter:
    Words: Counter = Counter()
    for file_name in os.listdir(directory):
        Words.update(get_common_words(directory + file_name))
    return Words

Next, we compute `Counter`s that store the number of occurrences in emails for every common word.

In [23]:
Spam_Counter: Counter = count_commmon_words(spam_dir_train)
Spam_Counter

Counter({'our': 223,
         's': 219,
         'free': 198,
         'please': 188,
         'email': 185,
         'mail': 179,
         'one': 168,
         'address': 166,
         'list': 166,
         'com': 160,
         'receive': 157,
         'http': 157,
         'us': 156,
         'send': 154,
         'day': 154,
         'information': 153,
         'remove': 150,
         'here': 146,
         'over': 146,
         'want': 145,
         'need': 145,
         'offer': 143,
         'money': 140,
         'name': 133,
         'call': 132,
         'order': 130,
         'include': 129,
         'nt': 127,
         'check': 126,
         'internet': 124,
         'best': 123,
         'work': 123,
         'site': 121,
         'follow': 118,
         'many': 117,
         'today': 116,
         'business': 114,
         'every': 114,
         'most': 112,
         'www': 110,
         'first': 110,
         'service': 108,
         'll': 107,
         'start': 106,
    

In [24]:
Ham__Counter: Counter = count_commmon_words(ham__dir_train)
Ham__Counter


Counter({'language': 241,
         'university': 201,
         's': 189,
         'information': 174,
         'linguistic': 170,
         'http': 137,
         'email': 136,
         'please': 133,
         'e': 131,
         'follow': 130,
         'fax': 130,
         'include': 130,
         'english': 125,
         'one': 125,
         'call': 119,
         'www': 116,
         'research': 116,
         'word': 114,
         'address': 113,
         'interest': 112,
         'send': 109,
         'edu': 105,
         'linguistics': 103,
         'paper': 100,
         'first': 100,
         'work': 99,
         'papers': 99,
         'conference': 99,
         'd': 94,
         'question': 92,
         'contact': 89,
         'page': 89,
         'de': 87,
         'provide': 86,
         'study': 85,
         'available': 85,
         'name': 84,
         'both': 83,
         'form': 83,
         'discussion': 81,
         'abstract': 81,
         'mail': 81,
         'number': 8

For every common word $w$  we compute the probability that $w$ occurs in a spam or ham email.  The formula for spam is:
 
 $$ P(w \in\texttt{Spam}) = \frac{\mbox{number of spam emails containing $w$}}{\mbox{number of all spam emails}} $$
 
 The formula for ham is similar:
 
 $$ P(w \in\texttt{Ham}) = \frac{\mbox{number of ham emails containing $w$}}{\mbox{number of all ham emails}} $$
 
 However, if we would use this formula, then a common word $w$ that, for some reason, hasn't yet occurred in any spam email, would have a probability of $0$ of occurring in spam email.  Hence, our classifier would never classify an email with the word $w$ as spam.  Since this cannot be right, we assume that there is one additional spam email that contains every common word.  
This approach is called
*Laplace smoothing* and it changes the formula for $P(w \in\texttt{Spam})$ as follows:
 
 $$ P(w \in\texttt{Spam}) = \frac{\mbox{number of spam emails containing $w$ + 1}}{\mbox{number of all spam emails + 1}} $$
 
Of course, the formula for $P(w \in\texttt{Ham})$ is changed in a similar way:

$$ P(w \in\texttt{Ham}) = \frac{\mbox{number of ham emails containing $w$ + 1}}{\mbox{number of all ham emails + 1}} $$

In [39]:
Spam_Probability: dict[str, float] = {}
Ham__Probability: dict[str, float] = {}
for w in Common_Words:
    Spam_Probability[w] = (Spam_Counter[w] + 1) / (no_spam + 1) 
    Ham__Probability[w] = (Ham__Counter[w] + 1) / (no_ham  + 1) 
Spam_Probability

{'conversation': 0.019943019943019943,
 'committee': 0.002849002849002849,
 'hurry': 0.02564102564102564,
 'txt': 0.03418803418803419,
 'love': 0.1111111111111111,
 'naturally': 0.011396011396011397,
 'nt': 0.3646723646723647,
 'wait': 0.14814814814814814,
 'coordinator': 0.002849002849002849,
 'begin': 0.09401709401709402,
 'company': 0.2934472934472934,
 'trash': 0.039886039886039885,
 'definitely': 0.05413105413105413,
 'perceive': 0.002849002849002849,
 'iii': 0.014245014245014245,
 'quebec': 0.002849002849002849,
 'additional': 0.14814814814814814,
 'univ': 0.002849002849002849,
 'reject': 0.005698005698005698,
 'virtual': 0.03133903133903134,
 'instruct': 0.045584045584045586,
 'enclose': 0.08262108262108261,
 'progress': 0.037037037037037035,
 'convert': 0.02564102564102564,
 'particle': 0.002849002849002849,
 'theart': 0.022792022792022793,
 'front': 0.05698005698005698,
 'stun': 0.03133903133903134,
 'throughout': 0.05982905982905983,
 'guide': 0.08547008547008547,
 'game': 0.

Let us check those common words that have a probability of at least $10\%$ to occur in a spam mail, but that have a probability of less than $1\%$ to occur in a ham mail.

In [26]:
{ w for w in Common_Words if Spam_Probability[w] > 0.1 and Ham__Probability[w] < 0.01 }

{'advertise',
 'advertisement',
 'amaze',
 'aol',
 'bonus',
 'bulk',
 'cash',
 'delete',
 'don',
 'dream',
 'earn',
 'financial',
 'freedom',
 'fresh',
 'fun',
 'hit',
 'hundred',
 'investment',
 'mailing',
 'profit',
 'purchase',
 'remove',
 'sale',
 'save',
 'secret',
 'security',
 'sell',
 'thousand',
 'toll',
 'wait',
 'yours',
 'yourself'}

For example, the probability that a spam email contains the word `'earn'` is about $15\%$, while the probability that it occurs in a ham email is $0.55\%$.

In [27]:
Spam_Probability['earn'], Ham__Probability['earn']

(0.14814814814814814, 0.005698005698005698)

For the word `'dollar'` the probability that a spam email contains this word is about $21\%$, while the probability that this word occurs in a ham email is less than $2\%$.

In [28]:
Spam_Probability['dollar'], Ham__Probability['dollar']

(0.21082621082621084, 0.019943019943019943)

Given a file name `fn`, the function `log_probabilities` returns a pair of numbers $(p_1, p_2)$ such that.  

In order to compute whether a mail is spam or ham we have to compute

$$\arg\max\limits_{C \in \{\text{Spam},\, \text{Spam}\}}  \left(\prod\limits_{i=1}^m P(f_i \;|\; C)\right) \cdot P(C) $$

Therefore, we have to check, whether

$$ \left(\prod\limits_{i=1}^m P(f_i \;|\; \texttt{Spam})\right) \cdot P(\texttt{Spam}) > \left(\prod\limits_{i=1}^m P(f_i \;|\; \texttt{Ham})\right) \cdot P(\texttt{Ham}) $$

holds. When implementing the formula 
$$\arg\max\limits_{C \in \mathcal{C}}  \left(\prod\limits_{i=1}^m P(f_i \;|\; C)\right) \cdot P(C) $$
we have to be careful, because a naive implementation will evaluate the product
$$\prod\limits_{i=1}^m P(f_i \;|\; C)$$
as the number $0$ due to numerical underflow.  The trick to compute this product is to remember that
$$ \ln(a \cdot b) = \ln(a) + \ln(b) $$
and to transform the product into a sum of logarithms.  As the logarithm is a monotone function, we have 

$$ \begin{array}{llcl}
  & \left(\prod\limits_{i=1}^m P(f_i \;|\; \texttt{Spam})\right) \cdot P(\texttt{Spam}) & > & \left(\prod\limits_{i=1}^m P(f_i \;|\; \texttt{Ham})\right) \cdot P(\texttt{Ham}) \\
  \Leftrightarrow \qquad &
  \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; \texttt{Spam})\bigr) + \ln\bigl(P(\texttt{Spam})\bigr) & > & \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; \texttt{Ham})\bigr) + \ln\bigl(P(\texttt{Ham}) \bigr)
  \end{array}
$$

The function `log_probabilities(fn)` takes a filename `fn` as its first argument and returns the pair 
$$ \left(\sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; \texttt{Spam})\bigr) + \ln\bigl(P(\texttt{Spam})\bigr),\quad
         \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; \texttt{Ham})\bigr) + \ln\bigl(P(\texttt{Ham}) \bigr) \right)$$
as its result.  It should be noted that these number are not really the logarithms of probabilites.  The reason is that
the formula for the probability of a class $C$ is
$$
\frac{\prod\limits_{i=1}^m P(f_i \;|\; C)}{P(f_1 \wedge \cdots \wedge f_m)} \cdot P(C)
$$
and we are not computing the denominator $P(f_1 \wedge \cdots \wedge f_m)$.  However, this denominator is the same for spam and ham and hence we don't need it when comparing the respective probabilities.  Therefore, what the function `log_probabilities`is really computing are pairs of *relative logarithmic probabilities*.

In [29]:
def log_probabilities(fn: str) -> tuple[float, float]:
    log_p_spam: float = math.log(spam_prior)
    log_p_ham:  float = math.log(ham__prior)
    words: set[str] = get_common_words(fn)
    for w in Common_Words:
        if w in words:
            log_p_spam += math.log(Spam_Probability[w])
            log_p_ham  += math.log(Ham__Probability[w])
    return (log_p_ham, log_p_spam)

Let us test this function with a ham email.  Due to privacy concerns, the emails have been reduced to lists of words, which have then been permuted.

In [30]:
!cat EmailData/ham-train/3-430msg1.txt || type EmailData/ham-train/3-430msg1.txt 

syntax textbook summary week ago ask reader share experience cowper haegeman syntax textbook here s here s summary response syntax textbook clear response cowper s book bite too many one favorable evaluation respondant read text manuscript hear anyone class far response haegeman s textbook mix excerpt paraphrase comment below five generally positive evaluation better most text wellreceive student full annoy mistake misprint quite happy complaint student harder radford most student prefer haegeman radford longwind condescend plain moronic student lucid funny wellorganize relatively happy generally understandable organize criticism tend introduce theoretical concept first justification later three mostly negative evaluation tend digress choose focus unclear example e g thetacriterion implicit argument early book casemark frequently exemplify ecm verb student unconvinc ing student unanimously despise incredibly unclear poorly organize recommend against positive side wellorganize reference

In [31]:
log_probabilities('EmailData/ham-train/3-430msg1.txt')

(-231.3859186980788, -265.04520620108457)

Clearly, the ham probability is bigger than the spam probability.  Next, we check the general performance of our approach.

## Evaluate Precision and Recall

In order to evaluate the performance of this algorithm, we need to define two new concepts: *precision* and *recall*.  Let us call the ham emails the *positives*, while the spam emails are called the *negatives*.  Then we define

- *true positives*: ham emails that are correctly classified as ham,
- *false positives*: spam emails that are falsely classified as ham,
- *true negatives*: spam emails that are correctly classified as spam,
- *false negatives*: ham emails that are falsely classified as spam.

The *precision* of the spam classifier is then defined as
$$ \texttt{precision} = \frac{\mbox{number of true positives}}{\mbox{number of true positives} + \mbox{number of false positives}} $$
Therefore, the *precision* measures the percentage of the ham emails in the set of all emails that are classified as ham. The *recall* of the spam classifier is defined as
$$ \texttt{recall} = \frac{\mbox{number of true positives}}{\mbox{number of true positives} + \mbox{number of false negatives}} $$
Therefore, the *recall* measures the percentage of those ham emails that are indeed classified as ham.

Usually, it is very important that the recall is high as we don't want to miss a ham email because our classifier has incorrectly classified it as a spam email.
On the other hand, having a high precision is not that important. After all, if $10\%$ of the emails offered to us as ham are, in fact, spam, we might tolerate this.  However, we would certainly not tolerate missing $10\%$ of our ham emails because they are incorrectly classified as spam.

The function `precission_recall` takes two directories as arguments: `spam_dir` is supposed to contain spam emails, while `ham_dir` contains ham emails.  It computes the **precision** and the **recall** of our spam classifier with respect to these test data.  

Since it would be quite bad when we misclassify a valid email as spam but we can tolerate an occasional spam email that gets through our filter, we add a third parameter $\theta$.  We will only classify an email as spam if
$$
                \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; \texttt{Spam})\bigr) + \ln\bigl(P(\texttt{Spam})\bigr) \; > \; 
\vartheta \cdot \sum\limits_{i=1}^m \ln\bigl(P(f_i \;|\; \texttt{Ham})\bigr) + \ln\bigl(P(\texttt{Ham}) \bigr)
$$
The third parameter $\vartheta$ is the logarithmic threshold. Later, we will use a logarithmic threshold of $0.8$.
To make things concrete, assume that
* The relative logarithmic probability $p_1$ of a mail being ham is $p_1 = -250$, while
* the relative logarithmic probability $p_2$ of this mail being spam is $p_2 = 200$.
As we have 
$$p_1 = -250 < -200 = p_2$$
we would classify this mail as spam if we would just compare the relative logarithmic probabilities.
However, if we use the logarithmic threshold of $0.8$ we have 
$$\vartheta \cdot p_1 = 0.8 \cdot (-250) = -200 = p_2 $$ 
and hence the inequality
$$\vartheta \cdot p_1 < p_2$$
would not be valid.  Therefore, we would conservatively classify the email as ham. 

In [32]:
𝜗 = 0.8
log_p_ham  = -250
log_p_spam = -200
log_p_spam  > log_p_ham

True

In [33]:
log_p_ham * 𝜗 

-200.0

In [34]:
log_p_spam  > 𝜗 * log_p_ham

False

In [35]:
def precission_recall(spam_dir: str, ham_dir: str, 𝜗: float) -> tuple[float, float, float]:
    TN: int = 0 # true negatives
    FP: int = 0 # false positives
    for email in os.listdir(spam_dir):
        log_p_ham, log_p_spam = log_probabilities(spam_dir + email)
        if log_p_spam > 𝜗 * log_p_ham:
            TN += 1
        else:
            print(email, log_p_ham, log_p_spam)
            FP += 1
    FN: int = 0 # false negatives
    TP: int = 0 # true positives
    for email in os.listdir(ham_dir):
        log_p_ham, log_p_spam = log_probabilities(ham_dir + email)
        if log_p_spam > 𝜗 * log_p_ham:
            FN += 1
            print(email, log_p_ham, log_p_spam)
        else:
            TP += 1
    precision: float = TP / (TP + FP)
    recall:    float = TP / (TP + FN)
    accuracy:  float = (TN + TP) / (TN + TP + FN + FP)
    return precision, recall, accuracy

If we use $\vartheta = 0.8$, then we have a *precision* of $95\%$, and a *total recall* of $100\%$ on the training set.

In [36]:
precission_recall(spam_dir_train, ham__dir_train, 0.80)

spmsgb143.txt -102.21694454357933 -85.15043065343859
spmsga16.txt -254.65486942240983 -214.91188775741875
spmsgc73.txt -146.7702589209136 -121.1161450407825
spmsga99.txt -268.815711086367 -235.87897757142593
spmsga164.txt -288.37579717022754 -234.73080707853399
spmsgc62.txt -46.39076813782138 -53.23570079988103
spmsgc90.txt -23.188904998584217 -21.796401611265313
spmsgc50.txt -144.4059802589142 -119.64980797198908
spmsgc83.txt -90.67570830279118 -88.82527625560228
spmsga3.txt -213.4441779277444 -179.74329259269695
spmsgc42.txt -93.31346471543833 -75.25648374406954
spmsgb164.txt -93.31346471543833 -75.25648374406954
spmsga26.txt -137.59541392025127 -124.0253295548933
spmsgc33.txt -542.2740666312525 -470.9463385572714
spmsgb56.txt -169.70768367756767 -138.32002791648395
spmsgb161.txt -85.11825279312276 -70.46638310273725
spmsgb160.txt -89.81695715397186 -73.63462331163687


(0.9536784741144414, 1.0, 0.9757142857142858)

For the test set, we still have a *recall* of $100\%$, but the *precision* drops to $92\%$, which is still acceptable.

In [37]:
precission_recall(spam_dir_test, ham__dir_test, 0.80)

spmsga12.txt -46.260934224410136 -40.543713470234714
spmsgc117.txt -332.6015272748802 -278.8736070207274
spmsgb156.txt -146.7702589209136 -121.1161450407825
spmsgb48.txt -365.784630178033 -294.15553780456526
spmsgc49.txt -144.4059802589142 -119.64980797198908
spmsgc52.txt -98.03931948482592 -79.75785827549348
spmsga150.txt -287.07651418609726 -233.7303008974281
spmsgb129.txt -103.66615124087335 -85.24641118832768
spmsgc18.txt -1602.680636028752 -1307.1641529585404
spmsgb90.txt -113.74089859017289 -106.15944634579304
spmsga125.txt -148.88391723458463 -133.44592411201623


(0.9219858156028369, 1.0, 0.9576923076923077)